# Basic Training Scheme development notebook

## Importing section

In [1]:
import os

import torch
from torch import optim, nn
from torch.utils.data import random_split, DataLoader
from etnn.data.ferris_wheel import load_pure_ferris_wheel_dataset
from etnn.data.tree_structure import TreeNode
from etnn.nn.layer_framework import ChiralLayerManagementFramework
from tqdm import tqdm
import matplotlib.pyplot as plt
from etnn.tools.training import train_epoch, eval_epoch, EpochControl

In [2]:
import numpy as np

## Parameter definition

In [3]:
NUM_GONDOLAS = 10
NUM_PART_PG = 5
DATASET_SIZE = 10_000
DATASET_PATH = "../datasets"

In [4]:
val_perc = 0.1
test_perc = 0.2
BATCH_SIZE = 32

In [5]:
INPUT_DIM = 15
HIDDEN_DIM = 32
OUT_DIM = 1
K=2
LEARNING_RATE = 0.001
NUM_MAX_EPOCHS = 10

In [6]:
MODEL_SAVING_PATH = ""
MODEL_SAVING_NAME = "model.pt"

## data preparation

In [7]:
dataset = load_pure_ferris_wheel_dataset(
    num_gondolas=NUM_GONDOLAS,
    num_part_pg=NUM_PART_PG,
    num_to_generate=DATASET_SIZE,
    dataset_path=DATASET_PATH
)

In [8]:
len(dataset)

10000

In [9]:
generator = torch.Generator().manual_seed(420)
train_ds, val_ds, test_ds = random_split(
    dataset,
    [1-val_perc-test_perc, val_perc, test_perc],
    generator=generator
)

In [10]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
# todo: add more tree mutated sequences that are in dataset
# todo: add some faulty sequences which disturb this and see how much this affects the training

## Build tree structure

In [12]:
tree_structure = TreeNode(
    node_type="C",
    children=[
        TreeNode("P", [TreeNode("E", NUM_PART_PG)])
        for _ in range(NUM_GONDOLAS)
    ]
)

## Define device

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
device

'cuda'

## Define Model

In [15]:
model = ChiralLayerManagementFramework(
    in_dim=15,
    tree=tree_structure,
    hidden_dim=HIDDEN_DIM,
    out_dim=OUT_DIM,
    k=K
).to(device)

## Define Loss and Optimizers

In [16]:
criterion = nn.MSELoss()

In [17]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

## Training

In [18]:
# init loss history
loss_history_train = []
loss_history_test = []

In [19]:
epoch_control = EpochControl(os.path.join(MODEL_SAVING_PATH, MODEL_SAVING_NAME))

In [20]:
# train for N epochs
for epoch in tqdm(range(NUM_MAX_EPOCHS)):
    mean_train_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        device,
        criterion
    )

    loss_history_train += [mean_train_loss]

    mean_test_loss = eval_epoch(
        model,
        test_loader,
        device,
        criterion
    )
    loss_history_test += [mean_test_loss]

    # check if model is better and save it
    epoch_control.retain_best(model, mean_train_loss, mean_test_loss)

 10%|█         | 1/10 [00:06<00:58,  6.48s/it]

++save++


 30%|███       | 3/10 [00:18<00:43,  6.24s/it]

++save++


 70%|███████   | 7/10 [00:42<00:18,  6.01s/it]

++save++


100%|██████████| 10/10 [01:00<00:00,  6.04s/it]


In [21]:
loss_history_train

[tensor(969.1465),
 tensor(39.2333),
 tensor(39.2346),
 tensor(39.1185),
 tensor(38.0965),
 tensor(38.7742),
 tensor(39.2909),
 tensor(38.3451),
 tensor(39.8947),
 tensor(39.8797)]

In [22]:
loss_history_test

[tensor(35.9496),
 tensor(36.0287),
 tensor(35.6956),
 tensor(35.9823),
 tensor(36.1628),
 tensor(36.0182),
 tensor(35.5542),
 tensor(40.4519),
 tensor(35.9143),
 tensor(37.6027)]

In [23]:
plt.figure()
idx = np.arange(len(loss_history_train))
offset = 1
plt.plot(idx[offset:], loss_history_train[offset:], label="train-loss")
plt.plot(idx[offset:], loss_history_test[offset:], label="test-loss")
plt.title("Loss plot")
plt.xlabel("Epoch")
plt.ylabel("mean loss value")
plt.legend()
plt.show()

## Test/Debug

for x,y in train_loader:
    break

x.shape

x

y.shape

y

for i in range(1000):
    optimizer.zero_grad()
    prediction = model(x).flatten()
    loss = criterion(prediction, y)
    print(f"Epoch:{i+1} - loss:{loss}")
    loss.backward()
    optimizer.step()

y

model(x).flatten()